In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

from nltk.tokenize.treebank import TreebankWordDetokenizer
%matplotlib inline

# Downloading and Preprocessing the Data

In [2]:
df = pd.read_csv('../data/ner_dataset.csv', encoding = "ISO-8859-1")
df.describe()

,Sentence #,Word,POS,Tag
count,47959,1048575,1048575,1048575
unique,47959,35178,42,17
top,Sentence: 10944,the,NN,O
freq,1,52573,145807,887908


In [3]:
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [4]:
print(df['Tag'].unique())

['O' 'B-geo' 'B-gpe' 'B-per' 'I-geo' 'B-org' 'I-org' 'B-tim' 'B-art'
 'I-art' 'I-per' 'I-gpe' 'I-tim' 'B-nat' 'B-eve' 'I-eve' 'I-nat']


In [5]:
df = df.fillna(method = 'ffill')

In [6]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [7]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [8]:
sent = getter.get_text()
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [9]:
sentences = getter.sentences

In [10]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [11]:
reduced_tag_set = ['B-geo', 'B-gpe', 'B-org', 'B-per', 'B-tim', 'I-geo',
                   'I-gpe', 'I-org', 'I-per', 'I-tim', 'O']

In [12]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
y = [[label if label in reduced_tag_set else 'O' for label in y_i] for y_i in y]  # reduce tag set

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.025)

In [14]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)

# GMB-Simple-Blurry

In [15]:
def blur_labels(y, frac=0.67):
    y_new = []
    error_array = []
    
    counter = 0
    
    for i in range(len(y)):
        error_array.append(list())
        y_new.append(list())
        change_steps = 0
        change_to = None
        
        for j in range(len(y[i])):
            
            current_tag = y[i][j]
            num_blur = np.random.randint(1,4)
            if current_tag=='B-geo' and  j >= num_blur and np.random.random() < frac:
                for k in range(num_blur):
                    y_new[i][j - k - 1] = current_tag
                    error_array[i][j - k - 1] = True
                
                y_new[i].append(current_tag)
                error_array[i].append(False)
            else:
                error_array[i].append(False)
                y_new[i].append(current_tag)
        
    return y_new, error_array

In [16]:
y_train_new, error_train_array = blur_labels(y_train)

In [17]:
t = sum([len(a) for a in error_train_array]); print('Num tags:', t)
e = sum([sum(a) for a in error_train_array]); print('Num errs:', e)
e/t

Num tags: 818841
Num errs: 33608


0.041043377163576325

In [18]:
print(len(X_train))

37407


In [19]:
crf.fit(X_train, y_train)
y_pred = crf.predict(X_test)
f1_score = flat_f1_score(y_test, y_pred, average = 'macro')
print(f1_score)

0.8449555980306126


In [20]:
report = flat_classification_report(y_test, y_pred, output_dict=True)
clean_precision, clean_recall, clean_f1 = report['B-geo']['precision'], report['B-geo']['recall'], report['B-geo']['f1-score'] 
print(flat_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-geo       0.87      0.91      0.89      7559
       B-gpe       0.96      0.94      0.95      3098
       B-org       0.82      0.75      0.78      4061
       B-per       0.85      0.83      0.84      3375
       B-tim       0.93      0.88      0.90      3994
       I-geo       0.84      0.81      0.83      1529
       I-gpe       0.78      0.53      0.63        40
       I-org       0.83      0.81      0.82      3491
       I-per       0.86      0.89      0.87      3426
       I-tim       0.82      0.76      0.79      1222
           O       0.99      0.99      0.99    177100

   micro avg       0.97      0.97      0.97    208895
   macro avg       0.87      0.83      0.84    208895
weighted avg       0.97      0.97      0.97    208895



In [21]:
crf.fit(X_train, y_train_new)
y_pred = crf.predict(X_test)
f1_score = flat_f1_score(y_test, y_pred, average = 'macro')
print(f1_score)

0.8082154282084513


In [22]:
report = flat_classification_report(y_test, y_pred, output_dict=True)
dirty_precision, dirty_recall, dirty_f1 = report['B-geo']['precision'], report['B-geo']['recall'], report['B-geo']['f1-score'] 
print(flat_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-geo       0.51      0.88      0.64      7559
       B-gpe       0.96      0.93      0.95      3098
       B-org       0.78      0.75      0.77      4061
       B-per       0.84      0.82      0.83      3375
       B-tim       0.92      0.86      0.89      3994
       I-geo       0.86      0.73      0.79      1529
       I-gpe       0.74      0.50      0.60        40
       I-org       0.80      0.82      0.81      3491
       I-per       0.85      0.88      0.87      3426
       I-tim       0.80      0.75      0.77      1222
           O       0.99      0.96      0.98    177100

   micro avg       0.95      0.95      0.95    208895
   macro avg       0.82      0.81      0.81    208895
weighted avg       0.96      0.95      0.95    208895



# Understanding the Mistakes

### Seeing the training data

In [23]:
for i in range(10):
    print("-", end="")
    for w, word in enumerate(X_train[i]):
        word = word['word.lower()']
        if error_train_array[i][w]:
            print('*', end='')            
        if y_train_new[i][w] == 'O':
            print(word.lower(), end=' ')
        else:
            print(word.upper(), end=' ')            
    print()

-JAPANESE carmaker TOYOTA MOTOR CORPORATION is reported to be considering letting FUJI HEAVY INDUSTRIES use its hybrid vehicle technology . 
-last month , PRESIDENT BUSH said the UNITED STATES could not rule out *USING *FORCE *IF IRAN fails to rein in its nuclear plans . 
-ISAF already *MAINTAINS *SECURITY *IN KABUL and the country 's north and west . 
-also in *NORTHERN *IRAQ *, KURDISH officials say search teams have located a small plane that crashed THREE days *AGO *NEAR SULAIMANIYAH . 
-the UNITED STATES says it looks forward to working with a new PAKISTANI government and PRESIDENT PERVEZ MUSHARRAF . 
-friends and family members of imprisoned opposition activists in BELARUS have sent an open letter to G-8 leaders , calling on them to press for the release of the activists . 
-settled as early as 1000 B.C. , SAMOA was " discovered " by european explorers in the 18TH CENTURY . 
-forecasters say hurricane otis is gaining strength as it *MOVES *CLOSER *TO MEXICO 's northwestern coast 

### Seeing the predictions on the test data

In [24]:
for i in range(10):
    print("-", end="")
    for w, word in enumerate(X_test[i]):
        word = word['word.lower()']
        if y_pred[i][w] != y_test[i][w]:
            print('*', end='')
            
        if y_pred[i][w] == 'O':
            print(word.lower(), end=' ')
        else:
            print(word.upper(), end=' ')            
    print()

-officials say MR. BUSH will also call for the further use of hybrid automobiles and those that burn clean diesel fuel by adding them to a list of vehicles eligible for tax credits . 
-REUTERS news agency reports a new indictment includes charges that *VANUNU , a christian convert , also violated release terms by trying to visit the WEST BANK in DECEMBER to attend a *MIDNIGHT CHRISTMAS religious service . 
-the action plan released by the JAPANESE health ministry MONDAY anticipates that a quarter of the population , some 32 million people , could become infected and that as many as 6,40,000 people could die . 
-the ECONOMIC COMMUNITY OF WEST AFRICAN STATES , ECOWAS , says it will not recognize the new government , calling the *EVENTS *IN TOGO a coup . 
-the ethnically fractured opposition failed to dislodge KANU from power in elections in 1992 AND 1997 , which were marred by violence and fraud , but were viewed as having generally reflected the will of the KENYAN people . 
-the group h

# Let's Try to Fix the Mistakes

For reference, the clean F1 is: **0.84** 

In [25]:
def measure_method(error_pred, error_array, X, y_corrected):
    # measure what percent of errors are fixed
    np = 0; nn=0; tp = 0; fp = 0;
    for i in range(len(error_pred)):
        for j in range(len(error_pred[i])):
            if error_pred[i][j] and error_pred[i][j] == error_array[i][j]:
                tp += 1
            elif error_pred[i][j]:
                fp += 1
            if error_array[i][j]:
                np += 1
            else:
                nn += 1

    
    print("TP errors detected: {}".format(tp/np))
    print("FP errors detected: {}".format(fp/nn)) 

    # measure accuracy
    crf.fit(X, y_corrected)
    y_pred = crf.predict(X_test)
    f1_score = flat_f1_score(y_test, y_pred, average = 'macro')
    print("F1 score on trained model: {}".format(f1_score))
    
    report = flat_classification_report(y_test, y_pred, output_dict=True)
    precision, recall, f1 = report['B-geo']['precision'], report['B-geo']['recall'], report['B-geo']['f1-score'] 
    print(flat_classification_report(y_test, y_pred))

    return precision, recall, f1

## Pseudolabeled from Validation

In [26]:
def pseudolabeled_on_validation(X_val, y_val):
    
    crf.fit(X_val, y_val)
    y_pred = crf.predict(X_train)

    error_pred = []
    y_corrected = []

    for i in range(len(y_pred)):    
        error_pred.append([])
        y_corrected.append([])

        for j in range(len(y_pred[i])):
            if not(y_pred[i][j]==y_train_new[i][j]):
                error_pred[i].append(True)
            else:
                error_pred[i].append(False)

    return measure_method(error_pred, error_train_array, X_train, y_pred)

### GTC with X, y, neighboring y

In [27]:
def gtc_with_x_y_neighboring_y(X_val, y_val, num_x_keys=1):
    keys_prioritized = ['word.isupper()', 'word.istitle()', 'word.isdigit()', '+1:word.istitle()', 
                        '+1:word.isupper()', 'BOS', 'bias',  'word.lower()', 'word[-3:]', 'word[-2:]',
                        '+1:word.lower()', 'postag', 'postag[:2]', '+1:postag', '+1:postag[:2]', 
                        '-1:postag', '-1:postag[:2]', '-1:word.istitle()', '-1:word.lower()',
                        '-1:word.isupper()', 'EOS']
    
    crf.fit(X_train, y_train_new)
    y_val_pred = crf.predict(X_val)

    correction_network_input = []

    for i in range(len(y_val_pred)):
        correction_network_input.append([])
        for j in range(len(y_val_pred[i])):
            if num_x_keys is None:
                X_val_sub = X_val[i][j].copy()
            else:
                X_val_sub = {k: X_val[i][j][k] for k in keys_prioritized[:num_x_keys] if k in X_val[i][j]}
            correction_network_input[i].append(X_val_sub)
            correction_network_input[i][j]['y'] = y_val_pred[i][j]     
            if j >= 1:
                correction_network_input[i][j]['y-1'] = y_val_pred[i][j-1]
            else:
                correction_network_input[i][j]['y-1'] = 'N'
            if j >= 2:
                correction_network_input[i][j]['y-2'] = y_val_pred[i][j-2]
            else:
                correction_network_input[i][j]['y-2'] = 'N'
            if j < len(y_val_pred[i]) - 1:
                correction_network_input[i][j]['y+1'] = y_val_pred[i][j+1]
            else:
                correction_network_input[i][j]['y+1'] = 'N'
            if j < len(y_val_pred[i]) - 2:
                correction_network_input[i][j]['y+2'] = y_val_pred[i][j+2]
            else:
                correction_network_input[i][j]['y+2'] = 'N'

    crf.fit(correction_network_input, y_val)

    X_expanded = []

    for i in range(len(y_train_new)):
        X_expanded.append([])
        for j in range(len(y_train_new[i])):
            if num_x_keys is None:
                X_train_sub = X_train[i][j].copy()
            else:
                X_train_sub = {k: X_train[i][j][k] for k in keys_prioritized[:num_x_keys] if k in X_train[i][j]}
            X_expanded[i].append(X_train_sub)
            X_expanded[i][j]['y'] = y_train_new[i][j]     
            if j >= 1:
                X_expanded[i][j]['y-1'] = y_train_new[i][j-1]
            else:
                X_expanded[i][j]['y-1'] = 'N'
            if j >= 2:
                X_expanded[i][j]['y-2'] = y_train_new[i][j-2]
            else:
                X_expanded[i][j]['y-2'] = 'N'
            if j < len(y_train_new[i]) - 1:
                X_expanded[i][j]['y+1'] = y_train_new[i][j+1]
            else:
                X_expanded[i][j]['y+1'] = 'N'
            if j < len(y_train_new[i]) - 2:
                X_expanded[i][j]['y+2'] = y_train_new[i][j+2]
            else:
                X_expanded[i][j]['y+2'] = 'N'

    # Go from X_expanded to X_corrected
    y_corrected = crf.predict(X_expanded)

    error_pred = []

    for i in range(len(y_corrected)):    
        error_pred.append([])    
        for j in range(len(y_corrected[i])):
            if not(y_corrected[i][j]==y_train_new[i][j]):
                error_pred[i].append(True)
            else:
                error_pred[i].append(False)

    return measure_method(error_pred, error_train_array, X_train, y_corrected)

### GTC with y, neighboring y only

In [28]:
def gtc_with_y_neighboring_y(X_val, y_val, num_ys=1):
    
    crf.fit(X_train, y_train_new)
    y_val_pred = crf.predict(X_val)

    correction_network_input = []

    for i in range(len(y_val_pred)):
        correction_network_input.append([])
        for j in range(len(y_val_pred[i])):
            correction_network_input[i].append(dict())
            correction_network_input[i][j]['y'] = y_val_pred[i][j] 
            for k in range(1, 1+num_ys):
                if j >= k:
                    correction_network_input[i][j]['y-{}'.format(k)] = y_val_pred[i][j-k]
                else:
                    correction_network_input[i][j]['y-{}'.format(k)] = 'N'
                if j < len(y_val_pred[i]) - k:
                    correction_network_input[i][j]['y+{}'.format(k)] = y_val_pred[i][j+1]
                else:
                    correction_network_input[i][j]['y+{}'.format(k)] = 'N'
    
    crf.fit(correction_network_input, y_val)

    X_expanded = []

    for i in range(len(y_train_new)):
        X_expanded.append([])
        for j in range(len(y_train_new[i])):
            X_expanded[i].append(dict())
            X_expanded[i][j]['y'] = y_train_new[i][j]     
            for k in range(1, 1+num_ys):
                if j >= k:
                    X_expanded[i][j]['y-{}'.format(k)] = y_train_new[i][j-k]
                else:
                    X_expanded[i][j]['y-{}'.format(k)] = 'N'
                if j < len(y_train_new[i]) - k:
                    X_expanded[i][j]['y+{}'.format(k)] = y_train_new[i][j+k]
                else:
                    X_expanded[i][j]['y+{}'.format(k)] = 'N'

    # Go from X_expanded to X_corrected
    y_corrected = crf.predict(X_expanded)

    error_pred = []

    for i in range(len(y_corrected)):    
        error_pred.append([])    
        for j in range(len(y_corrected[i])):
            if not(y_corrected[i][j]==y_train_new[i][j]):
                error_pred[i].append(True)
            else:
                error_pred[i].append(False)

    return measure_method(error_pred, error_train_array, X_train, y_corrected)

### How do results change based on markov blanket size?

In [29]:
n = 100
X_val_ = X_val[:n]
y_val_ = y_val[:n]

In [30]:
ppr, pre, pf1 = pseudolabeled_on_validation(X_val_, y_val_)

TP errors detected: 0.9588788383718162
FP errors detected: 0.06478331909127609


C:\Users\islam\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 score on trained model: 0.6001578145171726


C:\Users\islam\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       B-geo       0.66      0.85      0.74      7559
       B-gpe       0.87      0.72      0.79      3098
       B-org       0.52      0.46      0.49      4061
       B-per       0.80      0.51      0.62      3375
       B-tim       0.94      0.64      0.76      3994
       I-geo       0.59      0.42      0.49      1529
       I-gpe       0.00      0.00      0.00        40
       I-org       0.51      0.60      0.55      3491
       I-per       0.75      0.67      0.71      3426
       I-tim       0.83      0.32      0.46      1222
           O       0.98      0.99      0.98    177100

   micro avg       0.94      0.94      0.94    208895
   macro avg       0.68      0.56      0.60    208895
weighted avg       0.94      0.94      0.93    208895



In [31]:
pr, re, f1 = gtc_with_y_neighboring_y(X_val_, y_val_, num_ys=3)

TP errors detected: 0.7787431563913354
FP errors detected: 0.006353528188448524
F1 score on trained model: 0.7655365808212864
              precision    recall  f1-score   support

       B-geo       0.84      0.89      0.86      7559
       B-gpe       0.97      0.93      0.95      3098
       B-org       0.80      0.71      0.75      4061
       B-per       0.85      0.81      0.83      3375
       B-tim       0.93      0.85      0.89      3994
       I-geo       0.83      0.66      0.74      1529
       I-gpe       0.00      0.00      0.00        40
       I-org       0.75      0.79      0.77      3491
       I-per       0.84      0.90      0.87      3426
       I-tim       0.82      0.73      0.77      1222
           O       0.99      0.99      0.99    177100

   micro avg       0.97      0.97      0.97    208895
   macro avg       0.78      0.75      0.77    208895
weighted avg       0.97      0.97      0.97    208895



In [32]:
pr, re, f1 = gtc_with_x_y_neighboring_y(X_val_, y_val_, num_x_keys=None)

TP errors detected: 0.9390323732444656
FP errors detected: 0.0096238950731821
F1 score on trained model: 0.7626278975983044
              precision    recall  f1-score   support

       B-geo       0.83      0.91      0.87      7559
       B-gpe       0.96      0.93      0.95      3098
       B-org       0.80      0.70      0.75      4061
       B-per       0.87      0.79      0.83      3375
       B-tim       0.93      0.84      0.88      3994
       I-geo       0.83      0.68      0.75      1529
       I-gpe       0.00      0.00      0.00        40
       I-org       0.78      0.79      0.78      3491
       I-per       0.84      0.90      0.87      3426
       I-tim       0.85      0.62      0.72      1222
           O       0.99      0.99      0.99    177100

   micro avg       0.97      0.97      0.97    208895
   macro avg       0.79      0.74      0.76    208895
weighted avg       0.97      0.97      0.97    208895

